In [37]:
from dask.distributed import Client
import dask.dataframe as dd
import pandas as pd
import time
import statistics

import xgboost as xgb
import dask.array as da
import dask.distributed

In [38]:
times=[]

In [39]:
client = Client('172.31.45.128:8786')
client.restart()
print(client)

<Client: 'tcp://172.31.45.128:8786' processes=0 threads=0, memory=0 B>


In [40]:
data = dd.read_csv("/home/ubuntu/data.csv", sample=2**20)
data = data.persist()

In [41]:
train, test = data.random_split([0.7, 0.3], random_state=42)

In [42]:
X_train = train.drop(labels=["tip_amount"], axis=1).copy()
Y_train = train["tip_amount"].copy()

In [43]:
X_test = test.drop(labels=["tip_amount"], axis=1).copy()
Y_test = test["tip_amount"].copy()

# 10 Workers

In [44]:
t0 = time.time()
dtrain = xgb.dask.DaskDMatrix(client,X_train,Y_train)
output = xgb.dask.train(client
               ,{"verbosity": 2
                 , "tree_method": "hist"
                 , "objective": "reg:squarederror"}, dtrain
               , num_boost_round=100, evals=[(dtrain, "train")],)
t1 = time.time()
times.append(t1-t0)

In [45]:
t0 = time.time()
prediction = xgb.dask.predict(client, output['booster'], X_test)
t1 = time.time()
times.append(t1-t0)

In [46]:
import dask.array as da

# Assuming 'prediction' and 'Y_test' are defined properly
Y_test_array = Y_test.to_dask_array(lengths=True)

# Calculate Mean Squared Error
mse = da.mean((prediction - Y_test_array) ** 2)
rmse = da.sqrt(mse)

# Calculate Mean Absolute Error
mae = da.mean(da.absolute(prediction - Y_test_array))

# Calculate R^2 Score
ss_res = da.sum((prediction - Y_test_array) ** 2)
ss_tot = da.sum((Y_test_array - da.mean(Y_test_array)) ** 2)
r2_score = 1 - (ss_res / ss_tot)

# Compute the results
mse_result = mse.compute()
rmse_result = rmse.compute()
mae_result = mae.compute()
r2_score_result = r2_score.compute()

print(f"Mean Squared Error: {mse_result}")
print(f"Root Mean Squared Error: {rmse_result}")
print(f"Mean Absolute Error: {mae_result}")

Mean Squared Error: 0.13931918217669717
Root Mean Squared Error: 0.3732548488321313
Mean Absolute Error: 0.045213591681914966


In [47]:
times

[39.32284212112427, 0.15325307846069336]

# 6 Workers

In [48]:
t0 = time.time()
dtrain = xgb.dask.DaskDMatrix(client,X_train,Y_train)
output = xgb.dask.train(client
               ,{"verbosity": 2
                 , "tree_method": "hist"
                 , "objective": "reg:squarederror"}, dtrain
               , num_boost_round=100, evals=[(dtrain, "train")],)
t1 = time.time()
times.append(t1-t0)

In [49]:
t0 = time.time()
prediction = xgb.dask.predict(client, output['booster'], X_test)
t1 = time.time()
times.append(t1-t0)

In [50]:
Y_test_array = Y_test.to_dask_array(lengths=True)

# Calculate Mean Squared Error
mse = da.mean((prediction - Y_test_array) ** 2)
rmse = da.sqrt(mse)

# Calculate Mean Absolute Error
mae = da.mean(da.absolute(prediction - Y_test_array))

# Calculate R^2 Score
ss_res = da.sum((Y_test_array - prediction) ** 2)
ss_tot = da.sum((Y_test_array - da.mean(Y_test_array)) ** 2)
r2_score = 1 - (ss_res / ss_tot)

# Compute the results
mse_result = mse.compute()
rmse_result = rmse.compute()
mae_result = mae.compute()
r2_score_result = r2_score.compute()

print(f"Mean Squared Error: {mse_result}")
print(f"Root Mean Squared Error: {rmse_result}")
print(f"Mean Absolute Error: {mae_result}")

Mean Squared Error: 0.09078308288535643
Root Mean Squared Error: 0.3013023114504043
Mean Absolute Error: 0.04514299231963483


In [51]:
times

[39.32284212112427,
 0.15325307846069336,
 50.75175857543945,
 0.09682488441467285]

# 2 Workers

In [52]:
t0 = time.time()
dtrain = xgb.dask.DaskDMatrix(client,X_train,Y_train)
output = xgb.dask.train(client
               ,{"verbosity": 2
                 , "tree_method": "hist"
                 , "objective": "reg:squarederror"}, dtrain
               , num_boost_round=100, evals=[(dtrain, "train")],)
t1 = time.time()
times.append(t1-t0)

In [53]:
t0 = time.time()
prediction = xgb.dask.predict(client, output['booster'], X_test)
t1 = time.time()
times.append(t1-t0)

In [54]:
Y_test_array = Y_test.to_dask_array(lengths=True)

# Calculate Mean Squared Error
mse = da.mean((prediction - Y_test_array) ** 2)
rmse = da.sqrt(mse)

# Calculate Mean Absolute Error
mae = da.mean(da.absolute(prediction - Y_test_array))

# Calculate R^2 Score
ss_res = da.sum((Y_test_array - prediction) ** 2)
ss_tot = da.sum((Y_test_array - da.mean(Y_test_array)) ** 2)
r2_score = 1 - (ss_res / ss_tot)

# Compute the results
mse_result = mse.compute()
rmse_result = rmse.compute()
mae_result = mae.compute()
r2_score_result = r2_score.compute()

print(f"Mean Squared Error: {mse_result}")
print(f"Root Mean Squared Error: {rmse_result}")
print(f"Mean Absolute Error: {mae_result}")

Mean Squared Error: 0.09630681992805937
Root Mean Squared Error: 0.31033340124462816
Mean Absolute Error: 0.04493299284293645


In [55]:
times

[39.32284212112427,
 0.15325307846069336,
 50.75175857543945,
 0.09682488441467285,
 124.2763295173645,
 0.07560348510742188]